In [2]:
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import networkx as nx
import datetime as dt
import random
import pickle
import math
import csv
import os

%matplotlib inline

### Visualizations & plots

In [3]:
home_dir = os.path.expanduser("~")
code_dir = os.path.join(home_dir,"Developer","GitHub","endogenous-growth")
figs_dir = os.path.join(code_dir,"figures")

In [4]:
from utils import infomap_compression

### Network growth -- visualization

In [5]:
def node_properties(G_list):

    N = G_list[-1].number_of_nodes()
    m = G_list[0].number_of_nodes()
    
    # LAYOUT
    pos = nx.kamada_kawai_layout(G_list[0])
    pos_list = [pos.copy()] * m
    # now loop through the added nodes
    for node in range(m,len(G_list)):
        G = G_list[node]
        # get the new node into position
        tmp = nx.spring_layout(G, pos=pos, fixed=pos.keys())
        pos[max(G.nodes)] = tmp[max(G.nodes)]
        # get the new layout
        pos = nx.kamada_kawai_layout(G, pos=pos)
        pos_list.append(pos)
        
    # COLOR
    age_map = [i+1 for i in range(N)]
    node_ages = {i:1 for i in range(m)}
    age_list = [node_ages.copy()] * m
    # now loop through the added nodes
    for node in range(m,len(G_list)):
        G = G_list[node]
        node_ages[max(G.nodes)] = age_map[node]
        age_list.append(node_ages.copy())

    # SIZE
    pr_list = []
    for G in G_list:
        scores = nx.get_node_attributes(G, 'score')
        max_score = max(scores.values())
        scores_norm = {node: score/max_score for node, score in scores.items()}
        pr_list.append(scores_norm)

    return pos_list, age_list, pr_list


In [6]:
# NETWORK VIDEOS
runs = [1]
gammas = ["rnd",0,1,2,3,4,5,6,7,8,9,"inf"]

m = 3
alpha = 0.95

for run in runs:
    for gamma in gammas:
        
        network = "_".join(["i1o1","m"+str(m),"a"+str(alpha),"g"+str(gamma)])

        # LOAD RUN
        with open(os.path.join(code_dir,"networks",network,"run_"+str(run)+'.pkl'), 'rb') as f:
            model = pickle.load(f)

        # UNPACK
        networks = model.networks
        
        # READY OUTPUT DIRECTORY
        try:
            os.mkdir(os.path.join(code_dir,"networks",network,"pdfs_"+str(run)))
            os.mkdir(os.path.join(code_dir,"networks",network,"jpeg_"+str(run)))
        except:
            pass

        # NODE PROPERTIES
        pos_list, age_list, pr_list = node_properties(networks)
        
        # MAKE PLOTS
        for i, G in enumerate(networks):

            # initialize figure
            fig, ax = plt.subplots(1, 1, dpi=200, figsize=(4.5,4))

            # format node properties
            pos = pos_list[i]
            age = [age_list[i][node] for node in G.nodes]
            pr_rel = [10+200*pr_list[i][node] for node in G.nodes]

            # draw network
            nx.draw_networkx_nodes(G, pos, node_size=pr_rel, node_color=age, linewidths=1.5,
                                edgecolors='#333333', cmap='BuPu', ax=ax)
            nx.draw_networkx_edges(G, pos, edge_color='#999999', ax=ax, width=2, alpha=0.6)
            ax.set_axis_off()

            # draw colorbar
            cmap = plt.colormaps['BuPu']
            sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin = 1, vmax = i+1))
            sm.set_array([])
            plt.colorbar(sm, ax=ax)

            # save figure
            plt.savefig(os.path.join(code_dir,"networks",network,"jpeg_"+str(run),"N_"+str(G.number_of_nodes())+".jpeg"), bbox_inches='tight', dpi=300)
            plt.savefig(os.path.join(code_dir,"networks",network,"pdfs_"+str(run),"N_"+str(G.number_of_nodes())+".pdf"), bbox_inches='tight')
            plt.close()

In [16]:
# PICK YOUR FAVORITES
plot_gammas = [0,4,8]
fave_run = {0:0,4:0,8:1}
n_i = 25

plot_network = {}
plot_nodes = {}
for gamma in plot_gammas:
       
    # LOAD RUN
    network = "_".join(["i1o1","m"+str(m),"a"+str(alpha),"g"+str(gamma)])
    run = fave_run[gamma]

    with open(os.path.join(code_dir,"networks",network,"run_"+str(run)+'.pkl'), 'rb') as f:
        model = pickle.load(f)

    # UNPACK
    networks = model.networks
    plot_network[gamma] = networks[n_i-1]

    # NODE PROPERTIES
    pos_list, age_list, pr_list = node_properties(networks)
    plot_nodes[gamma] = {'pos':pos_list[n_i-1], 'age':age_list[n_i-1], 'pr_rel':pr_list[n_i-1]}

#initialize figure
fig, ax = plt.subplots(1, 3, dpi=200, figsize=(12,3.5), sharex=True, sharey=True)

# draw networks
for ax_i, gamma in enumerate(plot_gammas):

    # Grab network & node properties
    G = plot_network[gamma]
    pos = plot_nodes[gamma]['pos']
    age = [plot_nodes[gamma]["age"][node] for node in G.nodes]
    pr_rel = [10+200*plot_nodes[gamma]["pr_rel"][node] for node in G.nodes]

    nx.draw_networkx_nodes(G, pos, node_size=pr_rel, node_color=age, linewidths=1.5,
                        edgecolors='#333333', cmap='BuPu', ax=ax[ax_i])
    nx.draw_networkx_edges(G, pos, edge_color='#999999', ax=ax[ax_i], width=2, alpha=0.6)
    ax[ax_i].set_axis_off()
    ax_i += 1

# draw colorbar
cmap = plt.colormaps['BuPu']
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin = 1, vmax = n_i))
sm.set_array([])
fig.colorbar(sm, ax=ax.ravel().tolist())

ax[0].set_title("Random ("+r'$\gamma$'+f' = {plot_gammas[0]})', fontsize=14)
ax[1].set_title("Strategic ("+r'$\gamma$'+f' = {plot_gammas[1]})', fontsize=14)
ax[2].set_title("Near-optimal ("+r'$\gamma$'+f' = {plot_gammas[2]})', fontsize=14)

# READY OUTPUT DIRECTORY
output = "_".join(["i1o1","m"+str(m),"a"+str(alpha)])
if not os.path.exists(os.path.join(figs_dir,output)):
    os.mkdir(os.path.join(figs_dir,output))

plt.savefig(os.path.join(figs_dir,output,"networks.jpeg"), bbox_inches='tight', dpi=300)
plt.savefig(os.path.join(figs_dir,output,"networks.pdf"), bbox_inches='tight')
plt.close()



### Path dependence -- node trajectories

In [ ]:
# Example of a case where one link makes a big difference